In [1]:
#General imports needed for path
import os 
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

#import for plots
import matplotlib.pyplot as plt

In [9]:
import route_dynamics.route_elevation.base_df as base
import route_dynamics.route_energy.knn as knn
from route_dynamics.route_riders import route_riders as ride
import route_dynamics.route_energy.longi_dynam_model as ldm
import route_dynamics.route_energy.accel as ca

import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry import mapping

import warnings
warnings.filterwarnings('ignore')

In [12]:
#User defines what routes they want to evaluate
rt_list = [22]

#Import data files
shapefile_name = '../data/rt' + str(rt_list[0]) + '_pts2.shp'
stops_shp = '../data/Transit_Stops_for_King_County_Metro__transitstop_point.shp'

#Initialize route dataframe
rt_df = base.wrapper(shapefile_name, 6, 6)

In [5]:
rt_df.head()

,geometry,distance,elevation,speed_limit,grade
0,POINT (-122.38813 47.56295),0.0000,99.398090,11.175682,0.000000
1,POINT (-122.38798 47.56295),10.9728,99.874774,11.175682,0.043442
2,POINT (-122.38784 47.56295),21.9456,100.272354,11.175682,0.036233
3,POINT (-122.38769 47.56295),32.9184,100.594760,11.175682,0.029382
4,POINT (-122.38754 47.56295),43.8912,100.845922,11.175682,0.022889


In [6]:
# df, riders, mass = ride.route_ridership('AM', rt_list[0]) 

# stop_coord, rider_coord = ride.stop_coord(rt_list[0], riders)
# coords = rider_coord['coordinates'].values
# mass2 = rider_coord['Mean'].values

# stop_coords = stop_coord.coordinates.values

period = 'PM'
route = 22

In [7]:
trip183 = pd.read_csv("../data/Trip183.csv", usecols = ['SignRt', 'InOut', 'KeyTrip', 'BusType', 'Seats', 
                     'Period', 'AnnRides']) # KCM Data
trip183unsum = pd.read_csv("../data/Zon183Unsum.csv", usecols = ['Route', 'Dir', 'Trip_ID', 'InOut', 'STOP_SEQ', 'STOP_ID',
                     'Period', 'AveOn', 'AveOff', 'AveLd', 'Obs']) # KCM Data

In [10]:
trip_df = ride.route_ridership('PM', rt_list[0], 14000)
geometry = trip_df.geometry.values

In [11]:
xy = []
for i in range(len(geometry)):
    dic = mapping(geometry[i][0])
    coords = dic['coordinates']
    xy.append(coords)
    xy_df = pd.DataFrame(columns = ['STOP_ID','coordinates'])
    xy_df['coordinates'] = xy
    
xy_df

stop_coords = xy_df.coordinates.values

In [13]:
# stops = gpd.read_file(stops_shp)
# stops['ROUTE_LIST'].fillna(value=str(0), inplace=True)

In [14]:
# stops_list = pd.DataFrame()
# for i in range(0, len(stops)):

#     if str(rt_list[0]) in (stops['ROUTE_LIST'][i]):
#         for x in stops['ROUTE_LIST'][i].split(' '):
#             if str(rt_list[0]) == x:
#                 stops_list = stops_list.append(stops.iloc[i])
#             else:
#                 pass
#     else:
#         pass

# stop_ids = stops_list['STOP_ID'].values

# geometry = stops_list.geometry.values

# xy = []
# for i in range(len(geometry)):
#     dic = mapping(geometry[i])
#     coords = dic['coordinates']
#     xy.append(coords)
#     xy_df = pd.DataFrame(columns = ['STOP_ID','coordinates'])
#     xy_df['coordinates'] = xy

# xy_df['STOP_ID'] = stops_list['STOP_ID'].values

# df = riders_kept
# df_ind = df.reset_index()
# df_ind.columns = ['STOP_SEQ', 'STOP_ID', 'Mean']

# df_comb = xy_df.merge(df_ind, how='right', on="STOP_ID")
# df_comb = df_comb.drop_duplicates(subset=['STOP_ID'], keep='first')
# df_comb = df_comb[['STOP_SEQ', 'STOP_ID', 'coordinates', 'Mean']]
# df_combine = df_comb.sort_values(by='STOP_SEQ')

# df_combine.head()

In [29]:
# stop_coords = df_combine.coordinates.values

In [15]:
# add stops to bus dataframe

stop_nn_indicies, stop_coord_nn = knn.find_knn(
                1,
                rt_df.geometry.values,
                stop_coords
                )

In [16]:
np.shape(stop_nn_indicies)

(88, 1)

In [25]:
stop_nn_indicies.ravel()

array([ 615,  640,  656,  676,  694,  703,  722,  741,  748,  767,  784,
        802,  815,  831,  844,  864,  879,  895,  904,  914,  927,  939,
        955,  988, 1022, 1036, 1061, 1109, 1118, 1127, 1135, 1207, 1217,
       1247,  351, 1290,  320,  302, 1339,  268, 1362, 1372,  240, 1392,
       1413,  202,  190,  181,  169,  159, 1475,  134,  110, 1538,   61,
       1594, 1610, 1610,    7,   37,   64,   89, 1509,  139,  156, 1452,
       1440,  200,  211,  226, 1384,  248, 1364,  270,  285,  303,  319,
        334, 1270, 1246, 1220,  414, 1131, 1117,  502,  550,  571,  615])

In [26]:
rt_df = rt_df.assign(
                is_bus_stop = ([False] * len(rt_df.index))
                )

for i in stop_nn_indicies.ravel():
    rt_df.at[i, 'is_bus_stop'] = True

In [27]:
true_count = rt_df.is_bus_stop.sum()
true_count

86

In [28]:
rt_df.head()

,geometry,distance,elevation,speed_limit,grade,is_bus_stop
0,POINT (-122.38813 47.56295),0.0000,99.398090,11.175682,0.000000,False
1,POINT (-122.38798 47.56295),10.9728,99.874774,11.175682,0.043442,False
2,POINT (-122.38784 47.56295),21.9456,100.272354,11.175682,0.036233,False
3,POINT (-122.38769 47.56295),32.9184,100.594760,11.175682,0.029382,False
4,POINT (-122.38754 47.56295),43.8912,100.845922,11.175682,0.022889,False


In [30]:
mass_array = trip_df.Total_Mass.values

In [32]:
unloaded_bus_mass = 14000

In [33]:
# Initialize array of Nan's for mass column of rdf
full_mass_column = np.zeros(len(rt_df.index))
full_mass_column[:] = np.nan

# Iterate through the length of the given mass_array
# (already determined equal length to 'stop_coords').
for i in range(len(mass_array)):
    # Set values of mass at bus_stops
    full_mass_column[
        stop_nn_indicies[i]
        ] = mass_array[i]

# Set initial and value to unloaded bus mass.
full_mass_column[0] = unloaded_bus_mass
full_mass_column[-1] = unloaded_bus_mass

# Iterate through the half constructed rdf mass column
# ('full_mass_column') and fill in sapce between stops with previous value
for i in range(len(full_mass_column)-1):
    j = 1
    try:
        while np.isnan(full_mass_column[i+j]):
            full_mass_column[i+j] = full_mass_column[i]
            # print(full_mass_column[i+j] )
            j+=1
    except: IndexError

In [34]:
rt_df = rt_df.assign(
            mass = full_mass_column
            )

In [35]:
rt_df.head()

,geometry,distance,elevation,speed_limit,grade,is_bus_stop,mass
0,POINT (-122.38813 47.56295),0.0000,99.398090,11.175682,0.000000,False,14000.0
1,POINT (-122.38798 47.56295),10.9728,99.874774,11.175682,0.043442,False,14000.0
2,POINT (-122.38784 47.56295),21.9456,100.272354,11.175682,0.036233,False,14000.0
3,POINT (-122.38769 47.56295),32.9184,100.594760,11.175682,0.029382,False,14000.0
4,POINT (-122.38754 47.56295),43.8912,100.845922,11.175682,0.022889,False,14000.0


In [36]:
rt_df['mass'].value_counts()

14768.0    108
15536.0     86
14544.0     75
14352.0     75
14640.0     73
14712.0     72
14696.0     66
14512.0     64
14776.0     50
15160.0     49
14872.0     48
14424.0     45
14432.0     45
14800.0     44
14560.0     43
14664.0     41
14912.0     38
14496.0     38
14648.0     34
14608.0     34
14856.0     34
14456.0     29
14344.0     28
14920.0     27
14520.0     26
14360.0     26
14784.0     25
14624.0     23
15000.0     23
14744.0     21
14808.0     21
14928.0     21
14592.0     21
14448.0     17
14208.0     16
14576.0     15
15032.0     14
14632.0     14
14440.0     13
14952.0     12
14904.0     12
14656.0     12
14944.0     12
14296.0      9
14000.0      8
14816.0      5
14704.0      4
15344.0      3
15024.0      1
Name: mass, dtype: int64